## Süddeutsche Zeitung
#### Code for scraping Der Süddeutsche Zeitung 
- https://www.sueddeutsche.de

In [187]:
# Import necessary libraries
import requests
import urllib
import pandas as pd
from bs4 import BeautifulSoup

This site kindly uses a regular search string and allows you to search for a date range, so I scraped links from
the search page (50 per page) and then wrote a loop to scrape the text from each of those urls.

In [188]:
# Search term ("Flüchtling") and date range
search = '?search=Fl%C3%BCchtling&sort=date&all%5B%5D=dep&all%5B%5D=typ&all%5B%5D=sys&time=2021-03-16T00%3A00%2F2021-03-16T23%3A59&startDate=01.01.2015&endDate=31.12.2015'
# Site's base url
base_url = 'https://www.sueddeutsche.de/news/page/'
# results are organized by date within the range so I used multiples of 5 for the page numbers to try for 
# an even sample
n = 5
urls = []
while n <= 100:
    url = base_url+str(n)+search
    n += 5
    urls.append(url)
    
len(urls) #1000 potential articles

20

In [192]:
# Iterate through the urls of the search pages and get all the article urls embedded there 

art_links = []

for url in urls:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    divs = soup.find_all("div", {"class":"entrylist__entry"})
    #print(divs)
    for div in divs:
        a = div.find('a') 
        art_links.append(a.get('href'))
        
    
#art_links    

len(art_links)

1000

In [193]:
# Quick look to make sure everything seems to be working
art_links[:5]

['https://www.sueddeutsche.de/politik/migration-wams-ueber-8000-spezielle-deutschklassen-fuer-fluechtlinge-geschaffen-dpa.urn-newsml-dpa-com-20090101-151227-99-575735', 'https://www.sueddeutsche.de/politik/migration-italienische-kuestenwache-rettet-weitere-bootsfluechtlinge-dpa.urn-newsml-dpa-com-20090101-151226-99-574575', 'https://www.sueddeutsche.de/kultur/rueckblick-magische-kinomomente-des-jahres-2015-1.2785573', 'https://www.sueddeutsche.de/politik/rueckblick-2015-flucht-krieg-und-klimawandel-1.2794106', 'https://www.sueddeutsche.de/politik/fluechtlinge-schwaebisch-gmuend-will-zeichen-gegen-menschenfeindlichkeit-setzen-1.2796664']

In [194]:
# Go through all the collected urls and get the text and date, append to a dictionary for easy df making
date_dict = {}
art_dict_sz = {}
x = ' '
for link in art_links:
    #page = urllib.request.urlopen(link)
    response = requests.get(link)
    if response.status_code=='404': #in case it runs into any 404 messages
        print(url)
        pass
    soup = BeautifulSoup(response.text, 'html.parser')
    #soup = BeautifulSoup(page)
    t = soup.find("div", {"class":"css-1jpy2hx e1lg1pmy0"})
    if t==None:
        art_dict_sz[link] = 'None' # A few links are broken it seems
        pass
        #print(t)
    else:
        d = soup.find("time",  {"class":"css-1ccsr7y"})
        date = d.text
        paras = t.findAll('p')
        a = [p.text for p in paras]
        text = x.join(a)
        date_dict[link] = date
        art_dict_sz[link] = text
    
#art_dict_sz
#date_dict

In [195]:
# Lost a couple along the way, overall it still seems a good sample
len(art_dict_sz)
len(date_dict)

982

In [202]:
#Create a dataframe from the two dictionaries and add word count
df_sz = pd.DataFrame.from_dict(art_dict_sz, orient='index')
df_sz.reset_index(inplace=True)
df_sz.columns = ['href', 'text']
word_c = df_sz.text.str.split().map(len)
df_sz['word_count'] = word_c

df_sz['date']= df_sz['href'].map(date_dict)

df_sz.head()

,href,text,word_count,date
0,https://www.sueddeutsche.de/politik/migration-...,Berlin (dpa) - Die Bundesländer haben für die ...,89,"27. Dezember 2015, 2:45 Uhr"
1,https://www.sueddeutsche.de/politik/migration-...,Rom (dpa) - Im Mittelmeer vor Italien sind auc...,62,"26. Dezember 2015, 20:51 Uhr"
2,https://www.sueddeutsche.de/kultur/rueckblick-...,1 / 12 Quelle: 20th Century Fox Südseefilme si...,1818,"26. Dezember 2015, 17:57 Uhr"
3,https://www.sueddeutsche.de/politik/rueckblick...,Bei dem Blick zurück auf das Jahr 2015 stechen...,451,"26. Dezember 2015, 16:00 Uhr"
4,https://www.sueddeutsche.de/politik/fluechtlin...,Nach einem Brandanschlag auf eine noch nicht f...,387,"26. Dezember 2015, 15:43 Uhr"


In [204]:
# We have a few NaN and None types in this data frame
df_sz.info()
df_sz.head(30)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   href        1000 non-null   object
 1   text        1000 non-null   object
 2   word_count  1000 non-null   int64 
 3   date        982 non-null    object
dtypes: int64(1), object(3)
memory usage: 31.4+ KB


,href,text,word_count,date
0,https://www.sueddeutsche.de/politik/migration-...,Berlin (dpa) - Die Bundesländer haben für die ...,89,"27. Dezember 2015, 2:45 Uhr"
1,https://www.sueddeutsche.de/politik/migration-...,Rom (dpa) - Im Mittelmeer vor Italien sind auc...,62,"26. Dezember 2015, 20:51 Uhr"
2,https://www.sueddeutsche.de/kultur/rueckblick-...,1 / 12 Quelle: 20th Century Fox Südseefilme si...,1818,"26. Dezember 2015, 17:57 Uhr"
3,https://www.sueddeutsche.de/politik/rueckblick...,Bei dem Blick zurück auf das Jahr 2015 stechen...,451,"26. Dezember 2015, 16:00 Uhr"
4,https://www.sueddeutsche.de/politik/fluechtlin...,Nach einem Brandanschlag auf eine noch nicht f...,387,"26. Dezember 2015, 15:43 Uhr"
5,https://sz-magazin.sueddeutsche.de/texte/anzei...,None,1,NaN
6,https://www.sueddeutsche.de/politik/migration-...,Mainz (dpa) - CDU-Vizechefin Julia Klöckner hä...,87,"26. Dezember 2015, 12:12 Uhr"
7,https://www.sueddeutsche.de/service/jahreswech...,München (dpa) - Ifo-Präsident Hans-Werner Sinn...,378,"26. Dezember 2015, 11:51 Uhr"
8,https://www.sueddeutsche.de/leben/gesellschaft...,Hamburg (dpa) - Von der Aufnahme der zahlreich...,417,"26. Dezember 2015, 11:03 Uhr"
9,https://www.sueddeutsche.de/leben/gesellschaft...,"Darmstadt (dpa) - Zum ""Unwort des Jahres 2015""...",233,"26. Dezember 2015, 9:09 Uhr"


In [205]:
df_sz.describe()

,word_count
count,1000.000000
mean,368.440000
std,282.050192
min,1.000000
25%,119.000000
50%,331.500000
75%,532.250000
max,2402.000000


In [206]:
# Pickling the dataframe for easy use later, .gitignore is updated with .pkl files
pd.to_pickle(df_sz, "sz_df.pkl")